In [ ]:
import $ivy.`com.sksamuel.scrimage:scrimage-core_2.12:2.1.8`
import sys.process._
import javax.imageio.ImageIO
import java.io.File

In [ ]:
import $ivy.`io.circe:circe-core_2.12:0.10.1`
import $ivy.`io.circe:circe-generic_2.12:0.10.1`
import $ivy.`io.circe:circe-parser_2.12:0.10.1`
import _root_.io.circe.{Decoder, Encoder}

import io.circe.parser.decode

case class Detection(`class`: String, score: Float, box: (Int, Int, Int, Int))

object Detection {
  import _root_.io.circe.generic.semiauto._

  implicit lazy val encoder: Encoder[Detection] = deriveEncoder[Detection]
  implicit lazy val decoder: Decoder[Detection] = deriveDecoder[Detection]
}

In [ ]:
val imageFile = "resources/baywatch.jpg"

In [ ]:
def detectObjects(s: String) = {
    val cmd = s"""curl -X POST -F img=@${s}  localhost:8080/detect"""
    println(cmd)
    cmd.!!
}

In [ ]:
detectObjects(imageFile)

In [ ]:


def displayBoxes(imgFile: String, threshold: Float = 0.3f) = {
    val response = s"""curl -X POST -F img=@${imgFile}  localhost:8080/detect"""!!
    
    val detections = decode[Seq[Detection]](response) match {
        case Right(detections) => detections
        case Left(_) => List[Detection]()
    }
    
    val photo1 = ImageIO.read(new File(imgFile))
    val (width, height) = (photo1.getWidth, photo1.getHeight)
    def w(x: Int) = math.min(x, width-1)
    def h(x: Int) = math.min(x, height-1)
    
    detections.filter(_.score > threshold).foreach { d =>
    (d.box._1 until d.box._1 + d.box._3).map {x =>
        photo1.setRGB(w(x), h(d.box._2), 0xffffff)
        photo1.setRGB(w(x), h(d.box._2-1), 0xffffff)
        photo1.setRGB(w(x), h(d.box._2+1), 0xffffff)
        
        photo1.setRGB(w(x), h(d.box._2+d.box._4), 0xffffff)
        photo1.setRGB(w(x), h(d.box._2+d.box._4-1), 0xffffff)
        photo1.setRGB(w(x), h(d.box._2+d.box._4+1), 0xffffff)
    }
    (d.box._2 until d.box._2+ d.box._4).map {y =>
        photo1.setRGB(w(d.box._1), h(y), 0xffffff)
        photo1.setRGB(w(d.box._1-1), h(y), 0xffffff)
        photo1.setRGB(w(d.box._1+1), h(y), 0xffffff)
        photo1.setRGB(w(d.box._1 + d.box._3), h(y), 0xffffff)
        photo1.setRGB(w(d.box._1 + d.box._3-1), h(y), 0xffffff)
        photo1.setRGB(w(d.box._1 + d.box._3+1), h(y), 0xffffff)
    }
    }
    
    lazy val baos = new java.io.ByteArrayOutputStream()
    ImageIO.write( photo1, "png", baos );
    baos.flush()
    lazy val imageInByte = baos.toByteArray()
    baos.close()
    
    //Image.fromArray(imageInByte, Image.PNG,  width=Some("500"))
    Image.fromRenderedImage(photo1)// fv.withByteArray(imageInByte, Image.PNG,  width=Some("500"))
}

In [ ]:
detectObjects(imageFile)

In [ ]:
displayBoxes(imageFile, 0.9f)

In [ ]:
detectObjects("resources/lt.jpg")

In [ ]:
displayBoxes("resources/lt.jpg", 0.16F)

In [ ]:
displayBoxes("resources/thulsadoom.png", 0.16F)

In [ ]:
displayBoxes("resources/thulsadoomsnake.jpg", 0.16F)

In [ ]:
detectObjects("resources/thulsadoom.png")

In [ ]:
detectObjects("resources/thulsadoomsnake.jpg")

# Excercise

In [ ]:
Use the face detection model in place of the object detection model:

val modelname = "anonymizer"
val modelfilename = "weights_face_v1.0.0.pb"

